In [41]:
import pandas as pd
from sklearn import datasets
import nltk
# nltk.download('punkt') if not installed
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import re

In [2]:
train = pd.read_csv('dataset_train.csv')

In [3]:
train

,id,Tweets,Label
0,7281,The jokes and puns are flying free in this cam...,none
1,7282,#MKR Lets see who the producers think are goin...,none
2,7283,Praying Jac and Shaz do well! They're my faves...,none
3,7284,RT @realityraver: Pete Evans the Paleo Capital...,none
4,7285,If Kat and Andre stay tonight I will stop watc...,none
...,...,...,...
12824,5884,"RT @immichaeldixon: Katie and Nikki, smug, vac...",sexism
12825,15551,I can barely watch the #MKR episode of Katie a...,sexism
12826,15612,Gay fianc� is not going to cope being away fro...,sexism
12827,15638,#MKR you'd think in her downtime Annie would h...,sexism


In [4]:
train_x = train["Tweets"]
train_x 
train_y = train["Label"].factorize()[0]
train_y

array([0, 0, 0, ..., 2, 2, 2], dtype=int64)

In [5]:
# remove RT, mentions
train_x = train_x.replace("RT", "", regex=True).replace(r"@\b\S+", "", regex=True) #.replace(r"#\b", "", regex=True)
train_x

0        The jokes and puns are flying free in this cam...
1        #MKR Lets see who the producers think are goin...
2        Praying Jac and Shaz do well! They're my faves...
3          Pete Evans the Paleo Capitalist has had his ...
4        If Kat and Andre stay tonight I will stop watc...
                               ...                        
12824      Katie and Nikki, smug, vacuous, condescendin...
12825    I can barely watch the #MKR episode of Katie a...
12826    Gay fianc� is not going to cope being away fro...
12827    #MKR you'd think in her downtime Annie would h...
12828    � Really!!! Really!!! Another round!! #MKR tho...
Name: Tweets, Length: 12829, dtype: object

In [44]:
#stem train_x

train_x_tokenised = train_x.apply(nltk.word_tokenize)
train_x_tokenised

# https://www.datacamp.com/community/tutorials/stemming-lemmatization-python
#https://pythonhealthcare.org/2018/12/14/101-pre-processing-data-tokenization-stemming-and-removal-of-stop-words/
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

ps = PorterStemmer()
train_x = ps.stem(train_x)

AttributeError: 'Series' object has no attribute 'lower'

In [40]:
#from https://vgpena.github.io/classifying-tweets-with-keras-and-tensorflow/
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np

##obtain max
# create a new Tokenizer
tokenizer = Tokenizer()
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

#get the word counts so we can weed out words used less than x times
word_counts = pd.DataFrame.from_dict(tokenizer.word_counts, orient='index', columns=['count'])
word_counts

the          True
jokes        True
and          True
puns         True
are          True
            ...  
downtime    False
napol�on    False
perdis      False
learnt      False
blush       False
Name: count, Length: 17184, dtype: bool

In [39]:
#word_length = len(tokenizer.word_index)
#get words with at least 2 counts
max_words = sum(word_counts["count"]>1)
max_words

7578

In [8]:
# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x_processed = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y_processed = keras.utils.to_categorical(train_y)


c:\python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [9]:
#making the model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [10]:
model.fit(train_x_processed, train_y_processed,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Epoch 1/5
361/361 [==============================] - 29s 79ms/step - loss: 0.7028 - accuracy: 0.7430 - val_loss: 0.8867 - val_accuracy: 0.5931
Epoch 2/5
361/361 [==============================] - 28s 76ms/step - loss: 0.3184 - accuracy: 0.8789 - val_loss: 1.3039 - val_accuracy: 0.5230
Epoch 3/5
361/361 [==============================] - 31s 87ms/step - loss: 0.1739 - accuracy: 0.9392 - val_loss: 1.4730 - val_accuracy: 0.5900
Epoch 4/5
361/361 [==============================] - 37s 104ms/step - loss: 0.0860 - accuracy: 0.9738 - val_loss: 1.9637 - val_accuracy: 0.5503
Epoch 5/5
361/361 [==============================] - 40s 109ms/step - loss: 0.0443 - accuracy: 0.9861 - val_loss: 2.6759 - val_accuracy: 0.4942


In [11]:
#saving the file
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')